In [518]:
from flask import Flask, request, jsonify
import mysql.connector
import requests
import json

app = Flask(__name__)

#permet de se connecter à ma banque données
db = mysql.connector.connect(
    host="localhost", # l'herbergeur
    user='root', #l'utilisateur
    password='fLON111427!', #le mot de passe
    database='corentin_lgp_dm' #le nom de ma base de données
)
cursor = db.cursor() # permet d'initier un curseur

# Page d'acceuil quand on accède au serveur
@app.route('/')
def home_page():
    return 'Hello, Database!'



In [519]:
#permet d'accéder à toutes les tables
@app.route('/tables', methods=['GET']) # on précise la méthode à utiliser
def get_all_tables():
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    return jsonify(tables)

In [520]:
#permet d'accéder aux champs de la table phrase_originale
@app.route('/tables/phrase_originale', methods=['GET']) # on précise la méthode à utiliser
def get_content():
    cursor.execute("SELECT * FROM phrase_originale")
    phrase_originale = cursor.fetchall()
    return jsonify(phrase_originale)

In [521]:
#permet d'accéder aux champs de la table trad_auto
@app.route('/tables/trad_auto', methods=['GET']) # on précise la méthode à utiliser
def get_content_2():
    cursor.execute("SELECT * FROM trad_auto")
    trad_auto = cursor.fetchall()
    return jsonify(trad_auto)

In [522]:
#permet d'accéder aux champs de la table trad_hum
@app.route('/tables/trad_hum', methods=['GET']) # on précise la méthode à utiliser
def get_content_3():
    cursor.execute("SELECT * FROM trad_hum")
    trad_hum = cursor.fetchall()
    return jsonify(trad_hum)

In [523]:
#permet d'accéder aux champs de la table trad_auto en fonction de l'ID
@app.route('/tables/trad_auto/id/<int:trad_auto_id>', methods=['GET']) # on précise la méthode à utiliser
def get_data_trad_id(trad_auto_id):
    query = "SELECT * FROM trad_auto WHERE trad_auto_id = %s"
    cursor.execute(query, (trad_auto_id,))
    trad_auto_id = cursor.fetchone()
    if trad_auto_id:
        return jsonify(trad_auto_id)
    else:
        return jsonify({'error': 'not found'}), 404

In [524]:
#permet d'accéder aux champs de la table trad_hum en fonction de l'ID
@app.route('/tables/trad_hum/id/<int:trad_hum_id>', methods=['GET']) # on précise la méthode à utiliser
def get_data_tradhum_id(trad_hum_id):
    query = "SELECT * FROM trad_hum WHERE trad_hum_id = %s"
    cursor.execute(query, (trad_hum_id,))
    trad_hum_id = cursor.fetchone()
    if trad_hum_id:
        return jsonify(trad_hum_id)
    else:
        return jsonify({'error': 'not found'}), 404

In [525]:
#permet d'accéder aux champs de la table phrase_originale en fonction de l'ID
@app.route('/tables/phrase_originale/id/<int:phrase_id>', methods=['GET']) # on précise la méthode à utiliser
def get_data_by_id(phrase_id):
    query = "SELECT * FROM phrase_originale WHERE phrase_id = %s"
    cursor.execute(query, (phrase_id,))
    phrase_id = cursor.fetchone()
    if phrase_id:
        return jsonify(phrase_id)
    else:
        return jsonify({'error': 'not found'}), 404

In [512]:
#permet d'ajouter des données
@app.route('/add_data', methods=['POST']) # on précise la méthode à utiliser
def add_data():
    if request.method == 'POST':

        phrase_texte = request.form['phrase_texte'] # on fait des requêtes en focntion des champs
        texte_auto = request.form['texte_auto']  
        texte_hum = request.form['texte_hum']
        
        # on établit une connection
        try:
            connection = db
            cursor = connection.cursor()

            # on ajoute des données en fonction de la table, de la colonne et des valeurs à ajouter
            add_phrase_query = "INSERT INTO phrase_originale (phrase_texte) VALUES (%s)"
            cursor.execute(add_phrase_query, (phrase_texte,))
            
            add_data2_query = "INSERT INTO trad_auto (texte_auto) VALUES (%s)"
            cursor.execute(add_data2_query, (texte_auto,))

            add_data3_query = "INSERT INTO trad_hum (texte_hum) VALUES (%s)"
            cursor.execute(add_data3_query, (texte_hum,))

            connection.commit() # le commit s'assurer que les ajouts sont pris en compte
            cursor.close()
            connection.close()

            return 'Phrases Ajoutees' # message qui indique la réussite de la tâche

        except mysql.connector.Error as error: # on gère les erreurs
            return f'Error: {error}'

In [513]:
# chemin qui permet de mettre à jour les données
@app.route('/update_data/<int:item_id>', methods=['PUT']) # on précise la méthode à utiliser
def update_data(item_id):
    if request.method == 'PUT':
        try:
            # on fait des requêtes pour changer des valeurs
            phrase_texte = request.form['phrase_texte']  
            texte_auto = request.form['texte_auto']  
            texte_hum = request.form['texte_hum']  

            # on établit une connexion
            connection = db
            cursor = connection.cursor() # on ouvre un curseur

            # on fait la mise à jour pour chacune des tables
            update_phrase_query = "UPDATE phrase_originale SET phrase_texte = %s WHERE phrase_id = %s"
            cursor.execute(update_phrase_query, (phrase_texte, item_id))
            
            
            update_trad_auto_query = "UPDATE trad_auto SET texte_auto = %s WHERE trad_auto_id = %s"
            cursor.execute(update_trad_auto_query, (texte_auto, item_id))
            
            
            update_trad_hum_query = "UPDATE trad_hum SET texte_hum = %s WHERE trad_hum_id = %s"
            cursor.execute(update_trad_hum_query, (texte_hum, item_id))

            connection.commit() # on effectue un commit pour sauvegarder les changements
            cursor.close() # on ferme le curseur
            connection.close() # on ferme la connexion

            return 'Donnees Mises a Jour' # message qui indique la réussite de la tâche

        except mysql.connector.Error as error: # on gère les erreurs
            return f'Error: {error}'

In [516]:
# un chemin qui permet de supprimer des données
@app.route('/delete_data/<int:item_id>', methods=['DELETE']) # on précise la méthode à utiliser
def delete_data(item_id):
    try:
        # on établit une connexion 
        connection = db
        cursor = connection.cursor() # on ouvre un curseur

        # on supprime des données en fonction de l'id pour chaque table, colonne
        delete_phrase_query = "DELETE FROM phrase_originale WHERE phrase_id = %s"
        cursor.execute(delete_phrase_query, (item_id,))

        delete_trad_auto_query = "DELETE FROM trad_auto WHERE trad_auto_id = %s"
        cursor.execute(delete_trad_auto_query, (item_id,))

        delete_trad_hum_query = "DELETE FROM trad_hum WHERE trad_hum_id = %s"
        cursor.execute(delete_trad_hum_query, (item_id,))

        connection.commit() # on sauvegarde les changements
        cursor.close() # on ferme le curseur
        connection.close() # on ferme la connection

        return 'suppression réussie' # message qui indique la réussite de la tâche

    except mysql.connector.Error as error:
        return f'Error: {error}'

In [526]:
# permet de lancer le serveur une fois les blocs de chemin chargés
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2023 15:50:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2023 15:50:26] "GET /tables HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2023 15:50:30] "GET /tables/phrase_originale HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2023 15:50:36] "GET /tables/phrase_originale/id/5 HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2023 15:50:43] "GET /tables/trad_auto/id/5 HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2023 15:50:48] "GET /tables/trad_hum/id/5 HTTP/1.1" 404 -
